# Ligandos Cocristalizados para *Cross docking**

### Obtención una conformación minimizada de cada ligando cocristalizado con la proteína

**Esta libreta tiene como objetivo determinar cuántos ligandos únicos han sido cocristalizados con la proteína en el sitio activo y generar conformaciones minimizadas de dichos ligandos para posteriormente realizar un _cross docking_ con dichas moléculas y todas las conformaciones disponibles de la porteína.**

### Importamos las liberías y los datos necesarios

In [12]:
import pandas as pd
import glob, os

In [13]:
# Nombre de la proteina de interes
prot_name = 'cdk2'

Cargamos el dataFrame con los datos de los ligandos cocristalizados y generamos un conjunto que incluya los ligandos que han sido cocristalizados con la proteína (sin repetir elementos).

In [20]:
# Path hacia el archivo
patho_to_json_file = glob.glob((os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys_LIGS_INFO.json')))[0]
# Cargamos la tabla de datos
df_prot_labels = pd.read_json(patho_to_json_file)
# Obtenemos el nombre de los ligandos
lista_de_ligs_cdk2 = df_prot_labels.Inhib[df_prot_labels.Inhib.notnull()]
# Generamos un set para eliminar nombres repetidos
ligs_crys_prot = list(set(lista_de_ligs_cdk2))

In [18]:
print(F'Número de ligandos identificados: {len(ligs_crys_prot)}')

Número de ligandos identificados: 315


## Procesamos los ligandos y generamos los archivos

Definición y creación de los directorios de salida:

In [19]:
# Creamos el directorio donde guardaremos los ligandos
ligs_dir = '../ARCHIVOS/CRISTALES/LIGS_CDK2'
input_dir = F'{ligs_dir}/IN_POCKET_NON_PREP'
output_dir = F'{ligs_dir}/LIGS_PREP_CROSS_DK_317'

# Creamos el directorio de salida
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Usamos OpenBabel para generar los archivos mol2

**Nota:** Tener en cuenta que los archivos *mol2* generados por *babel* no dan conflictos con *prepare_ligand42.py* de ADTools, no obstante sí tienen conflictos con *rdkit*. Así que de ser necesario, generar archivos *sdf* también.

**Los ligandos serán generados añadiéndoles hidrógenos a un pH de 7.0 y usando el parámetro --gen3d para obtener una conformación 3D minimizada de los mismos.**

#### Gen3D
> To illustrate how some of the above methods might be used in practice, consider the gen3d operation. This operation (invoked using --gen3d at the commandline) generates 3D structures for 0D or 2D structures using the following series of steps, all of which have been described above:
- Use the OBBuilder to create a 3D structure using rules and fragment templates
- Do 250 steps of a steepest descent geometry optimization with the MMFF94 forcefield
- Do 200 iterations of a Weighted Rotor conformational search (optimizing each conformer with 25 steps of a steepest descent)
- Do 250 steps of a conjugate gradient geometry optimization

### Ejecutamos OpenBabel desde este notebook

In [26]:
%%bash -s "$input_dir" "$output_dir" "{' '.join(ligs_crys_prot)}" 
contador=0
for i in $3 # Para cada ligando en la lista
do 
contador=$((contador+1))
echo ${contador}\) Procesando ${i}...
# Convierte el archivo 
obabel -ipdb ${1}/*${i}*.pdb -omol2 -O ${2}/${i}.mol2 -c --gen3d -p 7.0
done

1) Procesando JYM...
2) Procesando 12Z...
3) Procesando N20...
4) Procesando LIA...
5) Procesando CK2...
6) Procesando 4CK...
7) Procesando EFQ...
8) Procesando 1PU...
9) Procesando BLZ...
10) Procesando T2A...


# ¡Terminado!